In [1]:
# create the configuration of the low-fidelity model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch


from mfbml.methods.dnn_lr_bnn import DNNLinearRegressionBNN as DNNLRBNN

# fix the random seed for reproducibility
seed = 1999
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
# read the data
data = pd.read_pickle("../data_generation/data.pkl")
# extract dataset
samples = data["samples"]
hf_samples = samples[0]
lf_samples = samples[1]
hf_responses = data["responses_lf1"][0]
responses_lf1 = data["responses_lf1"]
responses_lf2 = data["responses_lf2"]
responses_lf3 = data["responses_lf3"]
test_samples = data["test_samples"]
test_hf_responses_noiseless = data["test_hf_responses_noiseless"]
test_hf_responses = data["test_hf_responses"]
test_lf1_responses = data["test_lf1_responses"]
test_lf2_responses = data["test_lf2_responses"]
test_lf3_responses = data["test_lf3_responses"]

In [3]:
# create the configuration of the low-fidelity model
lf_configure = {
    "in_features": 1,
    "hidden_features": [50, 50],
    "out_features": 1,
    "activation": "Tanh",
    "optimizer": "Adam",
    "lr": 0.001,
    "weight_decay": 0.000001,
    "loss": "mse",
}
# create the configuration of the high-fidelity model
hf_parallel_configure = {
    "in_features": 1,
    "hidden_features": [512, 512],
    "out_features": 1,
    "activation": "Tanh",
    "lr": 0.001,
    "sigma": 0.05,
}

# lf train config
lf_train_config = {
    "batch_size": None,
    "num_epochs": 10000,
    "print_iter": 100,
    "data_split": False,
}
hf_train_config = {
    "num_epochs": 30000,
    "sample_freq": 100,
    "print_info": True,
    "burn_in_epochs": 20000,
}

In [4]:
# create the MFDNNBNN object
lf_responses = [responses_lf1, responses_lf2, responses_lf3]
for i, lf_response  in enumerate(lf_responses):
    model = DNNLRBNN(
        design_space=torch.Tensor([[0, 1]]),
        lf_configure=lf_configure,
        hf_configure=hf_parallel_configure,
        beta_optimize=True,
        lf_order=1,
        beta_bounds=[-5, 5],
        discrepancy_normalization="diff",
    )
    # define beta
    model.train(
        X=samples,
        Y=lf_response,
        lf_train_config=lf_train_config,
        hf_train_config=hf_train_config,
    )
    # save the model to the disk
    with open(f"dnn_lr_bnn_lf{i+1}_order_{i+1}_512_512_diff.pkl", "wb") as f:
        torch.save(model, f)


No data split: use all data for training
epoch:  100 train loss:  0.9584435820579529
epoch:  200 train loss:  0.9574099183082581
epoch:  300 train loss:  0.9541923403739929
epoch:  400 train loss:  0.9133488535881042
epoch:  500 train loss:  0.8133435845375061
epoch:  600 train loss:  0.5631825923919678
epoch:  700 train loss:  0.37910962104797363
epoch:  800 train loss:  0.0417165532708168
epoch:  900 train loss:  0.012330312281847
epoch:  1000 train loss:  0.006874925456941128
epoch:  1100 train loss:  0.0049567269161343575
epoch:  1200 train loss:  0.004295203369110823
epoch:  1300 train loss:  0.003997720777988434
epoch:  1400 train loss:  0.0038646592292934656
epoch:  1500 train loss:  0.003782189218327403
epoch:  1600 train loss:  0.003731832606717944
epoch:  1700 train loss:  0.00370135810226202
epoch:  1800 train loss:  0.003667067503556609
epoch:  1900 train loss:  0.003646103898063302
epoch:  2000 train loss:  0.003625324694439769
epoch:  2100 train loss:  0.00361136673018336

/home/yaga/Documents/GitHub/mfbml/src/mfbml/inference/psgld.py:108: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  V.mul_(alpha).addcmul_(1 - alpha, d_p, d_p)


epoch:   100/30000
nll_loss: 299.876, prior_loss: 273260.906, total: 273560.781
noise: 0.133
epoch:   200/30000
nll_loss: 276.606, prior_loss: 284524.000, total: 284800.594
noise: 0.133
epoch:   300/30000
nll_loss: 177.264, prior_loss: 294310.219, total: 294487.469
noise: 0.133
epoch:   400/30000
nll_loss: 136.271, prior_loss: 303446.156, total: 303582.438
noise: 0.133
epoch:   500/30000
nll_loss: 108.182, prior_loss: 312017.969, total: 312126.156
noise: 0.133
epoch:   600/30000
nll_loss: 80.448, prior_loss: 319809.812, total: 319890.250
noise: 0.133
epoch:   700/30000
nll_loss: 127.553, prior_loss: 327136.594, total: 327264.156
noise: 0.133
epoch:   800/30000
nll_loss: 89.609, prior_loss: 333935.156, total: 334024.750
noise: 0.133
epoch:   900/30000
nll_loss: 141.675, prior_loss: 340594.188, total: 340735.875
noise: 0.133
epoch:  1000/30000
nll_loss: 31.210, prior_loss: 346502.906, total: 346534.125
noise: 0.133
epoch:  1100/30000
nll_loss: 25.199, prior_loss: 352038.469, total: 35206

KeyboardInterrupt: 

In [ ]:
# load the model (one model for example)
with open("dnn_lr_bnn_lf1_order_1_512_512_diff.pkl", "rb") as f:
    model = torch.load(f)
# visualize the posterior of mf-dnn-bnn
(
    mfdnnbnn_lf1_hy,
    mfdnnbnn_lf1_epistemic,
    mfdnnbnn_lf1_total_unc,
    mfdnnbnn_lf1_aleatoric,
) = model.predict(x=test_samples)
# get lf predictions
mfdnnbnn_lf1_y = model.predict_lf(x=test_samples, output_format="numpy")

# plot
fig, ax = plt.subplots()
ax.plot(hf_samples, hf_responses, "o", label="hf")
ax.plot(test_samples.numpy(), mfdnnbnn_lf1_hy, label="hf prediction")
ax.plot(
    test_samples.numpy(),
    test_hf_responses_noiseless.numpy(),
    label="hf ground truth",
)
ax.fill_between(
    test_samples.flatten().numpy(),
    (mfdnnbnn_lf1_hy - 2 * mfdnnbnn_lf1_total_unc).reshape(-1),
    (mfdnnbnn_lf1_hy + 2 * mfdnnbnn_lf1_total_unc).reshape(-1),
    alpha=0.5,
    label="uncertainty",
)
# plot lf samples
ax.scatter(lf_samples, responses_lf1["lf"], label="LF samples")
ax.plot(test_samples.numpy(), mfdnnbnn_lf1_y,
        label="LF prediction")
ax.legend()
plt.show()